In [ ]:
import requests
import json
import pandas as pd
import time

API_KEY_FRESHWORK = ********
DOMAIN_FRESHWORK = 'institutadios'
BASE_URL = f'https://{DOMAIN_FRESHWORK}.myfreshworks.com/crm/sales/'

def get_existing_contacts(api_key, domain):
    headers = {
        'Content-Type': 'application/json',
        'Authorization': f'Token token={api_key}'
    }
    all_contacts = {}
    page = 1
    last_contact_id = None

    while True:
        params = {'per_page': 100, 'page': page}
        if last_contact_id:
            params['last_contact_id'] = last_contact_id
        url = f"{BASE_URL}api/contacts/view/31004512042?per_page=100&sort=created_at"

        for _ in range(3):  # Tentatives répétées en cas d'échec
            response = requests.get(url, headers=headers, params=params)
            if response.ok:
                break
            time.sleep(1)  # Pause avant de retenter

        if not response.ok:
            print(f"Erreur lors de la récupération des données : {response.status_code}")
            break

        response_data = response.json()
        if 'contacts' not in response_data:
            print("Erreur dans la structure de la réponse JSON")
            break

        contacts_data = response_data['contacts']
        if not contacts_data:
            break

        for contact in contacts_data:
            bitrix_id = contact['custom_field'].get('cf_id_bitrix')
            if bitrix_id:  # Vérifier si 'cf_id_bitrix' existe
                all_contacts[bitrix_id] = contact

        last_contact_id = contacts_data[-1]['id']
        page += 1
        time.sleep(1)  # Pause pour éviter le throttling

    print(f"Contacts récupérés avec succès : {len(all_contacts)}")
    return all_contacts

def main():
    existing_contacts = get_existing_contacts(API_KEY_FRESHWORK, DOMAIN_FRESHWORK)
    data = {
        "first_name": [],
        "last_name": [],
        "email": [],
        "work_number": [],
        "updated_at": []
    }
    custom_field_data = {}
    
    custom_fields = set()
    for contact in existing_contacts.values():
        custom_fields.update(contact.get("custom_field", {}).keys())
    
    for field in custom_fields:
        custom_field_data[field] = []
    for contact in existing_contacts.values():
        data["first_name"].append(contact.get("first_name", ""))
        data["last_name"].append(contact.get("last_name", ""))
        data["email"].append(contact.get("email", ""))
        data["work_number"].append(contact.get("work_number", ""))
        updated_at = contact.get("updated_at", "")
        if updated_at:
            updated_at = updated_at.split("T")[0]
        data["updated_at"].append(updated_at)
        custom_field = contact.get("custom_field", {})
        for field in custom_fields:
            custom_field_data[field].append(custom_field.get(field, ""))

    df_main = pd.DataFrame(data)
    df_custom = pd.DataFrame(custom_field_data)
    df_combined = pd.concat([df_main, df_custom], axis=1)
    
    df_combined["updated_at"] = pd.to_datetime(df_combined["updated_at"])
    df_combined["month_year"] = df_combined["updated_at"].dt.to_period('M')
    df_combined = df_combined.sort_values(by="month_year", ascending=False)
    df_combined = df_combined.drop(columns=["month_year"])
    df_combined = df_combined.reset_index(drop=True)
    
    df_combined.to_csv('contacts_rs.csv', index=False, encoding='utf-8')
    
    return df_combined

if __name__ == '__main__':
    contacts_df = main()
    print(contacts_df)


In [ ]:
import pandas as pd

doublonss = pd.read_csv("contacts_rs.csv")

doublonss["updated_at"] = pd.to_datetime(doublonss["updated_at"])

doublonss["month_year"] = doublonss["updated_at"].dt.to_period('M')

doublonss = doublonss.sort_values(by="month_year", ascending=False)

doublonss = doublonss.drop(columns=["month_year"])

doublonss = doublonss.reset_index(drop=True)

doublonss


In [ ]:
# Traitement des doublons par email

import pandas as pd

df_bitrixs = pd.read_csv("contacts_rs.csv", encoding='utf-8', low_memory=False)

df_filtre = df_bitrixs.dropna(subset=['email'])

doublons = df_filtre[df_filtre.duplicated(subset=['email'], keep=False)]

doublons = doublons.sort_values(by='email')

if doublons.empty:
    print("Aucun doublon trouvé basé sur les colonnes spécifiées.")
else:
    print(doublons)


In [ ]:
del doublons["cf_evenement_dacquisition"]
del doublons["cf_budget_journalier_utm_medium"]
del doublons["cf_webinar_time"]
del doublons["cf_webinar_date"]
del doublons["cf_a_assiste_au_live"]
del doublons["cf_inscription_utm_source"]
del doublons["cf_type_de_prospect"]
del doublons["cf_message_publicitaire_utm_content"]
del doublons["cf_nom_de_la_campagne_de_publicit_utm_campaign"]
del doublons["cf_note2"]
del doublons["cf_url_de_la_source"]
del doublons["cf_live_room_link"]
del doublons["cf_note1"]
del doublons["cf_date_de_naissance"]
del doublons["cf_canal_dacquisition"]
del doublons["cf_replay_link"]
del doublons["cf_time_zone"]
del doublons["cf_ciblage_utm_term"]
del doublons["cf_action_initiale_du_prospect"]
del doublons["cf_nom_du_prospect"]
del doublons["cf_objectif_formul"]
del doublons["cf_id_bitrix"]

doublons.to_csv("Listes_des_doublons_fresh.csv", index=False)
doublons

In [ ]:
#doublons par email

import pandas as pd

df_bitrixs = pd.read_csv("contacts_rs.csv", encoding='utf-8', low_memory=False)

df_filtre = df_bitrixs.dropna(subset=['email'])

doublons = df_filtre[df_filtre.duplicated(subset=['email'], keep=False)]

doublons = doublons.sort_values(by='email')

if doublons.empty:
    print("Aucun doublon trouvé basé sur les colonnes spécifiées.")
else:
    print(doublons)


In [ ]:
import gspread
import csv
from datetime import datetime

gc = gspread.service_account(filename="anomalieV2.json")

sh = gc.open_by_key('1cMEvZRMf8HWIeWIuZfGCojR4oMzPaJFGjnIEb4_4q8g')

worksheet = sh.worksheet("Anomalie 1-Doublons (email)")

with open("Listes_des_doublons_fresh.csv", 'r', encoding='utf-8') as f:
    reader = csv.reader(f)
    rows = list(reader)

last_col_num = ord('A') + len(rows[0]) - 1
last_col_letter = chr(last_col_num) if last_col_num <= ord('Z') else 'Z'

cell_range = f'A2:{last_col_letter}{len(rows)+1}'

cell_list = worksheet.range(cell_range)

for cell, value in zip(cell_list, [cell for row in rows for cell in row]):
    cell.value = value
worksheet.update_cells(cell_list)

now = datetime.now()
date_string = now.strftime("%d/%m/%Y à %H:%M:%S")
worksheet.update_acell('H2', date_string)

worksheet = None
